In [1]:
import json
import ast
import numpy as np
import pandas as pd
import nltk
import spacy
from spacy import displacy
from collections import Counter
import random;random.seed(1)
import en_core_web_sm
import textacy
import re
import unidecode
from text2digits import text2digits
t2d = text2digits.Text2Digits()
nlp = en_core_web_sm.load()


In [3]:
#with open('squad-dev-v1.1.json', 'r') as f:
    #df = json.load(f)
with open('wiki_squad_00-qa.json', 'r') as f:
    df = json.load(f)

In [90]:
#build dataframe for squad to get random sample of 100
topics = []
paragraphs = []
questions = []
qids = []
answers = []
autoq = pd.DataFrame()
for title in range(len(df['data'])):
    for p in range(len(df['data'][title]['paragraphs'])):
        if df['data'][title]['paragraphs'][p]['qas'] == []:
            topics.append(df['data'][title]['title'])
            paragraphs.append(df['data'][title]['paragraphs'][p]['context'])
            questions.append('')
            answers.append('')
            qids.append('')
        else:
            for qa in range(len(df['data'][title]['paragraphs'][p]['qas'])):
                topics.append(df['data'][title]['title'])
                paragraphs.append(df['data'][title]['paragraphs'][p]['context'])
                questions.append(df['data'][title]['paragraphs'][p]['qas'][qa]['question'])
                qids.append(df['data'][title]['paragraphs'][p]['qas'][qa]['id'])
                answers.append(df['data'][title]['paragraphs'][p]['qas'][qa]['answers'][0]['text'])
autoq['topics'] = topics
autoq['paragraphs'] = paragraphs
autoq['questions'] = questions
autoq['id'] = qids
autoq['answers'] = answers
autoq.head()
#sample100 = squad.sample(100, random_state = 5)
#new_index = [pd.Index(list(range(100)))]
#sample100.set_index(new_index, inplace=True)
#sample100.head()

,topics,paragraphs,questions,id,answers
0,Anarchism,Anarchism is a political philosophy that advoc...,,,
1,Anarchism,"While opposition to the state is central, many...",,,
2,Anarchism,Anarchism does not offer a fixed body of doctr...,,,
3,Anarchism,"The word ""anarchism"" is composed from the word...",,,
4,Anarchism,The first political philosopher to call himsel...,,,


In [91]:
def preprocess(answer):
    prep_answer = answer.lstrip('\'')
    prep_answer = prep_answer.rstrip('.')
    prep_answer = prep_answer.replace('\""','')
    prep_answer = prep_answer.replace('\"','')
    prep_answer = prep_answer.replace('\"','')
    prep_answer = prep_answer.lower()
    return prep_answer

In [92]:
def check_answer_type(raw_answer):
    answer_type = []
#     prep_answer = unidecode.unidecode(preprocess(raw_answer))
    tokenized_answer = nlp(raw_answer)
    pos_answer = [token.pos_ for token in tokenized_answer]
    month_list = ['january', 'jan', 'february', 'feb', 'march', 'april', \
                                   'may','june', 'july', 'august', 'aug', 'september', 'sept',\
                                   'october', 'oct', 'nov', 'dec','november', 'december']
    if (pos_answer[0]=='VERB') and (len(raw_answer)<2):
        answer_type.append('VERB')
    elif (pos_answer[0]=='VERB') and (len(raw_answer)>=2):
        answer_type.append('VP')
    # conditional for "to be" verbs
    elif (pos_answer[0]=='PART') and (pos_answer[1]=='VERB'):
        answer_type.append('VP')
    elif (pos_answer[0]=='ADV'):
        if len(tokenized_answer)>1:
            if (pos_answer[1]=='ADJ'):
                answer_type.append('ADJ')
            elif (pos_answer[1]=='VERB'):
                answer_type.append('VP')
            else:
                answer_type.append('ADV')
        else:
            answer_type.append('ADV')
    elif pos_answer[0]=='PROPN':
        # Check if it's a date
        if str(tokenized_answer[0]) in month_list:
            answer_type.append('DATE')
        else:
            answer_type.append('PROPN')
    elif (pos_answer[0]=='NUM') |  (pos_answer[0]=='PUNCT'):
        month_present = [1 for i in month_list if i in raw_answer]
        time_present = [1 for i in ['minutes', 'hours', 'seconds'] if i in raw_answer]
        if month_present!=[]:
            answer_type.append('DATE')
        elif time_present!=[]:
            answer_type.append('TIME')
        else:
            answer_type.append('NUM')
    elif pos_answer[0]=='ADJ':
        answer_type.append('ADJ')
    elif (pos_answer[0]=='NOUN') | (pos_answer[0]=='X'):
        # check if it's a hyphen-separated adjective
        if re.findall(r'\w+(?:-\w+)+'.lower(),raw_answer):
            answer_type.append('ADJ')
        # check if there's a digit in the answer
        elif bool(re.search(r'\d', raw_answer)):
             # check for currency symbols
            if bool(re.search(r'([£\$€])', raw_answer)):
                answer_type.append('MONEY')
            else:
                answer_type.append('YEAR')
        else:
            answer_type.append('NOUN')
    elif pos_answer[0]=='ADP':
        answer_type.append('ADP')
    elif pos_answer[0]=='DET':
        answer_type.append('NOUN')
    elif pos_answer[0]=='SYM':
        answer_type.append('MONEY')
    else:
        answer_type.append('Unknown type')
    return answer_type
    

In [112]:
def generate_distractor(topic, paragraph, question, answer, answer_type):
    wrong_answers=[]
    #preprocess answer
    correct_answer = nlp(preprocess(answer))
    # get answer pos
    ans_length = len(correct_answer)
    ans_tag = [token.tag_ for token in correct_answer]
    ans_pos = [token.pos_ for token in correct_answer]
    # tokenize paragraph
    article = nlp(unidecode.unidecode(paragraph))
    doc = textacy.Doc(paragraph, lang='en_core_web_sm')
        
    # Preprocessing for same sentence distractor generation
    # get all sentences in paragraph
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    sent_list = sent_detector.tokenize(paragraph.strip())

    # find sentence that has answer
    for s in sent_list:
        if answer in s:
            ans_sent = s
    # tokenize sentence
    sentence = nlp(ans_sent)
    sent = textacy.Doc(sentence, lang='en_core_web_sm')
        
    # Preprocessing for same topic distractor generation
    # choose random paragraph from same topic
    topic_index = next((index for (index, d) in enumerate(df['data']) if d["title"] == topic), None)
    index_list = list(range(len(df['data'][topic_index]['paragraphs'])))
    # make sure isn't the same paragraph as current paragraph
    p_index = next((index for (index, d) in enumerate(df['data'][topic_index]['paragraphs'])\
                    if d["context"] == paragraph), None)
    index_list.remove(p_index)
    alt_p_index = random.choice(index_list)
    alt_p = df['data'][topic_index]['paragraphs'][alt_p_index]['context']
    alt_article = nlp(alt_p)
    alt_paragraph = textacy.Doc(alt_p, lang='en_core_web_sm')
        
    # Preprocessing for different topic distractor generation
    # choose random topic
    topic_list = list(range(len(df['data'])))
    # make sure isn't the same topic as current topic
    topic_list.remove(topic_index)
    alt_topic_index = random.choice(topic_list)
    alt_topic = df['data'][alt_topic_index]['title']
    alt_topic_paragraph = df['data'][alt_topic_index]['paragraphs'][0]['context']
    # select first paragraph from topic - option to randomize selection as well
    alt_topic_article = nlp(alt_topic_paragraph)
    alt_topic = textacy.Doc(alt_topic_paragraph, lang='en_core_web_sm')
    
    ent_list = [str(i).lower() for i in list(article.ents)]
#     print('ent_list: ' + str(ent_list))

    if answer in ent_list:
#         print('in entity list')
        ent_labels = [x.label_ for x in article.ents]
        # get all named entities in sentence
        sent_ent_list = [str(i).lower() for i in list(sentence.ents)]
        sent_labels = [x.label_ for x in sentence.ents]
        alt_topic_ent_list = [str(i).lower() for i in list(alt_topic_article.ents)]
        alt_topic_labels = [x.label_ for x in alt_topic_article.ents]
        alt_article_ent_list = [str(i).lower() for i in list(alt_article.ents)]
        alt_article_labels = [x.label_ for x in alt_article.ents]
        merged=set(ent_labels+sent_ent_list+alt_topic_ent_list+alt_article_ent_list)
        # create table of named entities
        max_length = max(len(ent_list), len(sent_ent_list), len(alt_article_ent_list),len(alt_topic_ent_list))
#         print(max_length)
        ne_pd = pd.DataFrame()
        ne_pd['entity'] = ent_list + (['NA'] * (max_length - len(ent_list)))
        ne_pd['label'] = list(ent_labels) + (['NA'] * (max_length - len(ent_list)))
        ne_pd['sent_entity'] = sent_ent_list + (['NA'] * (max_length - len(sent_ent_list)))
        ne_pd['sent_label'] = list(sent_labels) + (['NA'] * (max_length - len(sent_ent_list)))
        ne_pd['altp_entity'] = alt_article_ent_list + (['NA'] * (max_length - len(alt_article_ent_list)))
        ne_pd['altp_label'] = list(alt_article_labels) + (['NA'] * (max_length - len(alt_article_ent_list)))
        ne_pd['altt_entity'] = alt_topic_ent_list + (['NA'] * (max_length - len(alt_topic_ent_list)))
        ne_pd['altt_label'] = list(alt_topic_labels) + (['NA'] * (max_length - len(alt_topic_ent_list)))

        ans_label = [ne_pd[ne_pd['entity']==e]['label'].values[0] for e in ne_pd['entity'] if e in answer]
#         print(ans_label)
        alt_ans_list = list(ne_pd[(ne_pd['label'].isin(ans_label))]['entity'])
        alt_ans_list.extend(list(ne_pd[(ne_pd['sent_label'].isin(ans_label))]['sent_entity']))
        alt_ans_list.extend(list(ne_pd[(ne_pd['altp_label'].isin(ans_label))]['altp_entity']))
        alt_ans_list.extend(list(ne_pd[(ne_pd['altt_label'].isin(ans_label))]['altt_entity']))
        wrong_answers = set(alt_ans_list)
        while len(wrong_answers)<3:
            index_list.remove(alt_p_index)
            alt_p_index = random.choice(index_list)
            alt_p = df['data'][topic_index]['paragraphs'][alt_p_index]['context']
            alt_article = nlp(alt_p)
            alt_paragraph = textacy.Doc(alt_p, lang='en_core_web_sm')
            alt_topic_ent_list = [str(i).lower() for i in list(alt_topic_article.ents)]
            alt_topic_labels = [x.label_ for x in alt_topic_article.ents]
            alt_article_ent_list = [str(i).lower() for i in list(alt_article.ents)]
            alt_article_labels = [x.label_ for x in alt_article.ents]
            merged=set(ent_labels+sent_ent_list+alt_topic_ent_list+alt_article_ent_list)
            # create table of named entities
            max_length = max(len(ent_list), len(sent_ent_list), len(alt_article_ent_list),len(alt_topic_ent_list))
    #         print(max_length)
            ne_pd = pd.DataFrame()
            ne_pd['entity'] = ent_list + (['NA'] * (max_length - len(ent_list)))
            ne_pd['label'] = list(ent_labels) + (['NA'] * (max_length - len(ent_list)))
            ne_pd['sent_entity'] = sent_ent_list + (['NA'] * (max_length - len(sent_ent_list)))
            ne_pd['sent_label'] = list(sent_labels) + (['NA'] * (max_length - len(sent_ent_list)))
            ne_pd['altp_entity'] = alt_article_ent_list + (['NA'] * (max_length - len(alt_article_ent_list)))
            ne_pd['altp_label'] = list(alt_article_labels) + (['NA'] * (max_length - len(alt_article_ent_list)))
            ne_pd['altt_entity'] = alt_topic_ent_list + (['NA'] * (max_length - len(alt_topic_ent_list)))
            ne_pd['altt_label'] = list(alt_topic_labels) + (['NA'] * (max_length - len(alt_topic_ent_list)))

            ans_label = [ne_pd[ne_pd['entity']==e]['label'].values[0] for e in ne_pd['entity'] if e in answer]
    #         print(ans_label)
            alt_ans_list = list(ne_pd[(ne_pd['label'].isin(ans_label))]['entity'])
            alt_ans_list.extend(list(ne_pd[(ne_pd['sent_label'].isin(ans_label))]['sent_entity']))
            alt_ans_list.extend(list(ne_pd[(ne_pd['altp_label'].isin(ans_label))]['altp_entity']))
            alt_ans_list.extend(list(ne_pd[(ne_pd['altt_label'].isin(ans_label))]['altt_entity']))
            wrong_answers = set(alt_ans_list)
        
    else:
#         print('not an entity')
        merged=[]
        while len(merged)<3:
            index_list.remove(alt_p_index)
            alt_p_index = random.choice(index_list)
            alt_p = df['data'][topic_index]['paragraphs'][alt_p_index]['context']
            alt_article = nlp(alt_p)
            alt_paragraph = textacy.Doc(alt_p, lang='en_core_web_sm')
            correct_ans_pos = str(['r'+str(token.pos_)+'l' for token in \
               nlp(answer)])[1:-1].replace("'r","<").replace("l'",">").replace(',','+',1).replace(',','*').replace(' ','')+'*'
#             print('correct_ans_pos: ' + str(correct_ans_pos))
            doc_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(doc, correct_ans_pos)]
#             print('doc_list: ' + str(doc_list))
            sent_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(sent, correct_ans_pos)]
#             print('sent_list: ' + str(sent_list))
            alt_paragraph_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_paragraph, correct_ans_pos)]
#             print('alt_paragraph_list: ' + str(alt_paragraph_list))
            alt_topic_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_topic, correct_ans_pos)]
#             print('alt_topic_list: ' + str(alt_topic_list))
            merged = set(doc_list+sent_list+alt_paragraph_list+alt_topic_list)
        
#         print('merged: ' + str(merged))
        if 'VP' in answer_type:
            if len(wrong_answers) >= 3:
                for i in merged:
                    item = nlp(i)
                    if (item[0].tag_ == ans_tag[0]) and (item[-1].pos_ == ans_pos[-1]):
                        wrong_answers.append(item)
            else:
                wrong_answers = merged
        if 'VERB' in answer_type:
            if len(wrong_answers) >= 3:
                for i in merged:
                    item = nlp(i)
                    if (item[0].tag_ == ans_pos_tag[0]):
                        wrong_answers.append(item)
            else:
                wrong_answers = merged
        if 'ADJ' in answer_type:
            hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),paragraph)
            hyphen_sep_words_sent = re.findall(r'\w+(?:-\w+)+'.lower(),ans_sent)
            alt_p_hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),alt_p)
            alt_topic_hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),alt_topic_paragraph)

            hyphen_words = hyphen_sep_words+hyphen_sep_words_sent+alt_p_hyphen_sep_words+alt_topic_hyphen_sep_words
#             print('hyphen_words: ' + str(hyphen_words))
            for w in hyphen_words:
                merged.add(w)
#             print('hyphen added merged: ' + str(merged))
            wrong_answers = merged
        if 'YEAR' in answer_type:
            for i in merged:
                if len(i)==4:
                    wrong_answers.append(i)
        if 'NUM' in answer_type:
            wrong_answers = [t2d.convert(str(i)).lstrip() for i in merged]
            if len(answer)==4:
                wrong_answers = [i for i in wrong_answers if len(i)==4]
        if 'TIME' in answer_type:
            wrong_answers = [t2d.convert(str(i)).lstrip()+' ' + str(correct_answer[-1]) for i in merged]
        if 'MONEY' in answer_type:
            currency = answer[0]
            print('currency: ' + str(currency))
            wrong_answers = [str(currency) + t2d.convert(str(i)).replace(' ','') for i in merged]
            print('wrong_answers: ' + str(wrong_answers))
        if 'DATE' in answer_type:
            if ans_length==1:
                month_list=random.sample(['January', 'Jan', 'February', 'March', 'April', \
                                       'May','June', 'July', 'August', 'September', \
                                       'October','November', \
                                          'December'].remove(answer.capitalize()),3)
        if 'ADP' in answer_type:
            wrong_answers = [i for i in wrong_answers if i[0]==correct_answer[0].orth_]
        if ('PROPN' in answer_type) | ('NOUN' in answer_type):
            ans_ent_list = [str(i).lower() for i in list(nlp(answer).ents)]
            ans_labels = [x.label_ for x in nlp(answer).ents]
            if ans_ent_list!=[]:
                for a in merged:
                    a_ent_list = [str(i).lower() for i in list(nlp(a.title()).ents)]
                    a_labels = [x.label_ for x in nlp(a.title()).ents]
                    ne_pd = pd.DataFrame()
                    ne_pd['ent'] = a_ent_list
                    ne_pd['label'] = a_labels
                    if nlp(a):
                        wrong_answers.append(list(ne_pd[(ne_pd['label'].isin(a_labels))]['ent']))
                wrong_answers=[item for sublist in wrong_answers for item in sublist]
            else:
                wrong_answers=merged
        else:
            wrong_answers=merged
    wrong_answers = [i for i in wrong_answers for word in range(len(correct_answer)) if (correct_answer[word].orth_ not in i)]
    wrong_answers=[item for item in wrong_answers if item not in ['a','an','the','that','it','who','what', '',""]]

    return (topic, paragraph, question, correct_answer, answer_type, random.sample(list(set(wrong_answers)),3))
       
                    

In [113]:
distractors = []
for i in range(len(autoq)):
    try:
        print('Sample: ' + str(i))
        print('Raw Answer: ' + str(autoq.loc[i]['answers']))
        answer_type = check_answer_type(autoq.loc[i]['answers'])
#         print('Answer type: ' + str(answer_type))
        topic, paragraph, question, correct_answer, answer_type, wrong_answers = generate_distractor(autoq.loc[i]['topics'], autoq.loc[i]['paragraphs'], \
                                  autoq.loc[i]['questions'], autoq.loc[i]['answers'], answer_type)
        distractors.append(wrong_answers)
        print('added distractors: ' + str(wrong_answers))
#         print('---------------------------------------------------------')
#         print('Question: ' + str(question))
#         print('Wrong answers: ' + str(wrong_answers))
#         print('\n')
    except:
        print("error occurred for sample#: " + str(i))
        distractors.append(['NA'])
autoq['distractors']=distractors
autoq['distractor_length'] = [1 if (len(autoq.loc[i]['distractors'])==1) else 0 for i in list(autoq.index.values) ]


Sample: 0
Raw Answer: 
error occurred for sample#: 0
Sample: 1
Raw Answer: 
error occurred for sample#: 1
Sample: 2
Raw Answer: 
error occurred for sample#: 2
Sample: 3
Raw Answer: 
error occurred for sample#: 3
Sample: 4
Raw Answer: 
error occurred for sample#: 4
Sample: 5
Raw Answer: the apostles
added distractors: ['an example', 'the organisations', 'a set']
Sample: 6
Raw Answer: anarchist
added distractors: ['word', 'nation', 'terms']
Sample: 7
Raw Answer: 
error occurred for sample#: 7
Sample: 8
Raw Answer: 
error occurred for sample#: 8
Sample: 9
Raw Answer: 
error occurred for sample#: 9
Sample: 10
Raw Answer: Proudhon
added distractors: ['accusation', 'abuse', 'set']
Sample: 11
Raw Answer: 
error occurred for sample#: 11
Sample: 12
Raw Answer: 
error occurred for sample#: 12
Sample: 13
Raw Answer: revolutionary anarchist programme
added distractors: ['italian anarchists', 'far-right', 'first critical theorists']
Sample: 14
Raw Answer: Anarchists
added distractors: ['result', 'l

added distractors: ['rare mutations', 'co-occur', 'like traits']
Sample: 131
Raw Answer: 
error occurred for sample#: 131
Sample: 132
Raw Answer: 
error occurred for sample#: 132
Sample: 133
Raw Answer: 
error occurred for sample#: 133
Sample: 134
Raw Answer: 
error occurred for sample#: 134
Sample: 135
Raw Answer: 
error occurred for sample#: 135
Sample: 136
Raw Answer: 
error occurred for sample#: 136
Sample: 137
Raw Answer: 
error occurred for sample#: 137
Sample: 138
Raw Answer: 
error occurred for sample#: 138
Sample: 139
Raw Answer: 
error occurred for sample#: 139
Sample: 140
Raw Answer: 
error occurred for sample#: 140
Sample: 141
Raw Answer: 
error occurred for sample#: 141
Sample: 142
Raw Answer: hyperbaric
added distractors: ['his', 'good', 'dominant']
Sample: 143
Raw Answer: Rain Man
added distractors: ['test results', 'others', 'oscars']
Sample: 144
Raw Answer: rights movement
added distractors: ['astronomy', 'studies', 'someone']
Sample: 145
Raw Answer: increasing the awa

added distractors: ['george orwell', 'moscow', 'august']
Sample: 267
Raw Answer: 
error occurred for sample#: 267
Sample: 268
Raw Answer: 
error occurred for sample#: 268
Sample: 269
Raw Answer: 
error occurred for sample#: 269
Sample: 270
Raw Answer: 
error occurred for sample#: 270
Sample: 271
Raw Answer: 
error occurred for sample#: 271
Sample: 272
Raw Answer: 
error occurred for sample#: 272
Sample: 273
Raw Answer: 
error occurred for sample#: 273
Sample: 274
Raw Answer: 
error occurred for sample#: 274
Sample: 275
Raw Answer: 
error occurred for sample#: 275
Sample: 276
Raw Answer: 
error occurred for sample#: 276
Sample: 277
Raw Answer: 
error occurred for sample#: 277
Sample: 278
Raw Answer: 
error occurred for sample#: 278
Sample: 279
Raw Answer: 
error occurred for sample#: 279
Sample: 280
Raw Answer: 31,180
added distractors: ['24,000', '73,811', '108,000']
Sample: 281
Raw Answer: 
error occurred for sample#: 281
Sample: 282
Raw Answer: 4,779,736
added distractors: ['310,000'

added distractors: ['life', 'leaders', 'part']
Sample: 393
Raw Answer: 
error occurred for sample#: 393
Sample: 394
Raw Answer: 
error occurred for sample#: 394
Sample: 395
Raw Answer: 
error occurred for sample#: 395
Sample: 396
Raw Answer: King Telephus
added distractors: ['ammianus marcellinus', 'telephus', 'american']
Sample: 397
Raw Answer: 
error occurred for sample#: 397
Sample: 398
Raw Answer: 
error occurred for sample#: 398
Sample: 399
Raw Answer: 
error occurred for sample#: 399
Sample: 400
Raw Answer: Homer's "Iliad
added distractors: ['niger,', 'tunisia,', 'mediterranean sea.']
Sample: 401
Raw Answer: 
error occurred for sample#: 401
Sample: 402
Raw Answer: Hector
added distractors: ['%', 'illusions', 'dsm-5']
Sample: 403
Raw Answer: 
error occurred for sample#: 403
Sample: 404
Raw Answer: 
error occurred for sample#: 404
Sample: 405
Raw Answer: 
error occurred for sample#: 405
Sample: 406
Raw Answer: 
error occurred for sample#: 406
Sample: 407
Raw Answer: 
error occurred

added distractors: ['fort sumter', 'united states', 'supreme court']
Sample: 520
Raw Answer: 
error occurred for sample#: 520
Sample: 521
Raw Answer: Frémont's proclamation
added distractors: ['johnson', 'frémont', 'john c. frémont']
Sample: 522
Raw Answer: by releasing the two diplomats
added distractors: ['with governors', 'in the audience', 'for balance']
Sample: 523
Raw Answer: James G. Randall
added distractors: ['trent affair', 'lincoln', 'great britain']
Sample: 524
Raw Answer: 
error occurred for sample#: 524
Sample: 525
Raw Answer: 
error occurred for sample#: 525
Sample: 526
Raw Answer: 
error occurred for sample#: 526
Sample: 527
Raw Answer: 
error occurred for sample#: 527
Sample: 528
Raw Answer: 
error occurred for sample#: 528
Sample: 529
Raw Answer: 
error occurred for sample#: 529
Sample: 530
Raw Answer: 
error occurred for sample#: 530
Sample: 531
Raw Answer: 
error occurred for sample#: 531
Sample: 532
Raw Answer: 
error occurred for sample#: 532
Sample: 533
Raw Answe

...

added distractors: ['orwell', 'apollo', 'hittite']
Sample: 1420
Raw Answer: Greek people
added distractors: ['oracular-cult', 'its death', 'complex']
Sample: 1421
Raw Answer: 
error occurred for sample#: 1421
Sample: 1422
Raw Answer: 
error occurred for sample#: 1422
Sample: 1423
Raw Answer: 
error occurred for sample#: 1423
Sample: 1424
Raw Answer: 
error occurred for sample#: 1424
Sample: 1425
Raw Answer: Dorians
added distractors: ['hand', 'years', 'bandwidth']
Sample: 1426
Raw Answer: 
error occurred for sample#: 1426
Sample: 1427
Raw Answer: 
error occurred for sample#: 1427
Sample: 1428
Raw Answer: 
error occurred for sample#: 1428
Sample: 1429
Raw Answer: 
error occurred for sample#: 1429
Sample: 1430
Raw Answer: 
error occurred for sample#: 1430
Sample: 1431
Raw Answer: 
error occurred for sample#: 1431
Sample: 1432
Raw Answer: 
error occurred for sample#: 1432
Sample: 1433
Raw Answer: 
error occurred for sample#: 1433
Sample: 1434
Raw Answer: 
error occurred for sample#: 1434


added distractors: ['embodied', 'is sanctified', 'man']
Sample: 1546
Raw Answer: Orestes
added distractors: ['temple', 'testimony', '/kʷalyán-/']
Sample: 1547
Raw Answer: Greeks
added distractors: ['founder', 'reign', 'play']
Sample: 1548
Raw Answer: 
error occurred for sample#: 1548
Sample: 1549
Raw Answer: 
error occurred for sample#: 1549
Sample: 1550
Raw Answer: 
error occurred for sample#: 1550
Sample: 1551
Raw Answer: bow and arrow
added distractors: ['turned', 'assessed', 'are']
Sample: 1552
Raw Answer: 
error occurred for sample#: 1552
Sample: 1553
Raw Answer: 
error occurred for sample#: 1553
Sample: 1554
Raw Answer: gluttony
added distractors: ['spoils', 'games', 'moderation']
Sample: 1555
Raw Answer: 
error occurred for sample#: 1555
Sample: 1556
Raw Answer: 
error occurred for sample#: 1556
Sample: 1557
Raw Answer: 
error occurred for sample#: 1557
Sample: 1558
Raw Answer: 
error occurred for sample#: 1558
Sample: 1559
Raw Answer: 
error occurred for sample#: 1559
Sample: 1

added distractors: ['mon', 'earth', 'harry l. shorto']
Sample: 1673
Raw Answer: 
error occurred for sample#: 1673
Sample: 1674
Raw Answer: 
error occurred for sample#: 1674
Sample: 1675
Raw Answer: 2015
added distractors: ['today', '1929', 'annually']
Sample: 1676
Raw Answer: 
error occurred for sample#: 1676
Sample: 1677
Raw Answer: 
error occurred for sample#: 1677
Sample: 1678
Raw Answer: 
error occurred for sample#: 1678
Sample: 1679
Raw Answer: 
error occurred for sample#: 1679
Sample: 1680
Raw Answer: 
error occurred for sample#: 1680
Sample: 1681
Raw Answer: 
error occurred for sample#: 1681
Sample: 1682
Raw Answer: 
error occurred for sample#: 1682
Sample: 1683
Raw Answer: 
error occurred for sample#: 1683
Sample: 1684
Raw Answer: 
error occurred for sample#: 1684
Sample: 1685
Raw Answer: 
error occurred for sample#: 1685
Sample: 1686
Raw Answer: 
error occurred for sample#: 1686
Sample: 1687
Raw Answer: 
error occurred for sample#: 1687
Sample: 1688
Raw Answer: 
error occurred

added distractors: ['hamito', 'afro', 'west asia']
Sample: 1807
Raw Answer: eastern Pyrenees mountain range
added distractors: ['basque', 'second century', 'official language']
Sample: 1808
Raw Answer: 
error occurred for sample#: 1808
Sample: 1809
Raw Answer: snow
added distractors: ['country', 'orientation', 'valleys']
Sample: 1810
Raw Answer: 
error occurred for sample#: 1810
Sample: 1811
Raw Answer: 
error occurred for sample#: 1811
Sample: 1812
Raw Answer: 
error occurred for sample#: 1812
Sample: 1813
Raw Answer: 
error occurred for sample#: 1813
Sample: 1814
Raw Answer: 
error occurred for sample#: 1814
Sample: 1815
Raw Answer: 
error occurred for sample#: 1815
Sample: 1816
Raw Answer: 
error occurred for sample#: 1816
Sample: 1817
Raw Answer: 
error occurred for sample#: 1817
Sample: 1818
Raw Answer: 
error occurred for sample#: 1818
Sample: 1819
Raw Answer: Andorra
added distractors: ['òscar ribas reig', 'executive board', 'european economic community']
Sample: 1820
Raw Answer

added distractors: ['canonbury square', 'trojan war', 'george orwell']
Sample: 1931
Raw Answer: 
error occurred for sample#: 1931
Sample: 1932
Raw Answer: 
error occurred for sample#: 1932
Sample: 1933
Raw Answer: 
error occurred for sample#: 1933


In [95]:
autoq.head()

,topics,paragraphs,questions,id,answers,distractors,distractor_length
0,Anarchism,Anarchism is a political philosophy that advoc...,,,,[NA],1
1,Anarchism,"While opposition to the state is central, many...",,,,[NA],1
2,Anarchism,Anarchism does not offer a fixed body of doctr...,,,,[NA],1
3,Anarchism,"The word ""anarchism"" is composed from the word...",,,,[NA],1
4,Anarchism,The first political philosopher to call himsel...,,,,[NA],1


In [114]:
wikipedia_data = {"data": [], "version" : 1.0}
print(wikipedia_data)
for topic in autoq.topics.unique():
    print('topic: ' + str(topic))
    unique_paragraphs = autoq[autoq['topics']==topic].paragraphs.unique()
    print('number of paragraphs: ' + str(len(unique_paragraphs)))
    agg = autoq[autoq['topics']==topic].groupby('paragraphs').agg({'questions': lambda x: '\n\n'.join(x), \
                                                                   'answers': lambda x: '\n'.join(x), \
                                                                   'id':lambda x: '\n'.join(x), \
                                                                  'distractors': lambda a: list(a)})
    context = []
    for item in range(len(unique_paragraphs)):
        para = unique_paragraphs[item]
        qas = []
        questions = agg.iloc[item]['questions'].split('\n\n')
        print('question')
        print(agg.iloc[item]['questions'])
        print('distractors')
        print(agg.iloc[item]['distractors'])
        for q in range(len(questions)):
            q_dict = {'question': '', 'id': '', 'answers': [], 'distractors': []}
            q_dict['question'] = questions[q]
            q_dict['id'] = agg.iloc[item]['id'].split('\n')[q]
            ans_dict = {'answer_start': 0, 'text': ''}
            ans_dict['text']= agg.iloc[item]['answers'].split('\n')[q]
            q_dict['answers'].append(ans_dict)
            qas.append(q_dict)
            q_dict['distractors'] = agg.iloc[item]['distractors'][q]
        context_dict = {'context': para, 'qas' : qas}
        context.append(context_dict)
    wikipedia_data['data'].append({'title' : topic, 'paragraphs' : context})

with open('wiki_squad_00-mc.json', 'w') as outfile:  
    json.dump(wikipedia_data, outfile)

{'data': [], 'version': 1.0}
topic: Anarchism
number of paragraphs: 84
question





distractors
[['NA'], ['NA'], ['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
in what year was the international of conference held ?

distractors
[['1960s', '1970', 'the 1917 october revolution']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
when was the uruguayan anarchist federation founded ?


when was the mexican anarchist federation established ?


when was the league of free social constructors formed ?


when was the dissolved federation formed ?

distractors
[['1953', '1947', 'in 1945'], ['from 1944 to 1947', '1956', '1955'], ['paris', 'antifascist international solidarity', 'communist federation'], ['february 1946', '1957', 'may 1948']]
question
what type of programme does imier congress adopt ?

distractors
[['italian anarchists', 'far-right', '

number of paragraphs: 145
question





distractors
[['NA'], ['NA'], ['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
what is the highest poverty rate in the us ?

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
what is the name of the region that the northwestern lies of ?

distractors
[['ayn rand', 'occupational education', 'u.s.']]
question
what was the population of michigan in 2010 ?

distractors
[['310,000', 'two', 'almost 800']]
question

distractors
[['NA']]
question
what university has the university of michigan ?

distractors
[['16th', 'late 20th', 'constitutional']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
how many counties has the ohio legislature has ?

distractors
[['NA']]
question

dist

[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
what was the nickname given to lincoln ?


when was robert todd lincoln born ?


who was the last ancestor of the lincolns' last ancestor ?


when was thomas `` tad '' born ?

distractors
[['robert todd lincoln beckwith', 'lee', 'ancient'], ['1985', '1862', 'february 1, 1850'], ['december', 'robert todd lincoln', 'april'], ['kentucky', 'lincoln', 'edward baker lincoln']]
question

distractors
[['NA']]
question
who james james j james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james e. james ?

distractors
[['august', 'eric foner', 'john c. frémont']]
questi

question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
how long did the first jaws last ?

distractors
[['for', 'of over 8,000', 'with']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
how many categories of films were awarded in the first year of the awards award ?

distractors
[['the year', '1998', '7 april 1970']]
question
what is the best known award for the academy award of merit ?

distractors
[['instant runoff voting', 'best visual', 'best films']]
question
how long have academy awards been ended ?

distractors
[['3', '1885', '28']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
how many honorary awards does the academy awards have ?

distractors
[['up to five', 'around 6,000', 'ten']]
question
where did maurice go to paris ?

distra


distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
who was the younger son of commercial traveller ?

distractors
[['monitor lizards', 'celluloid sheets', 'animals']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
what is algeria a regional and middle power power ?

distractors
[['the 20th century', 'january 2016', '2016']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distracto

question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
who translated latin into latin into latin into latin ?

distractors
[['capitalism', 'bodies', 'democracy']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
what did st. augustine declare in the 5th and 9th centuries ?

distractors
[['stylistic reasons', '12th', 'common practice']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
who was the author of the translation and preservation of greek texts ?

distractors
[['buddhist', 'nityanātha siddha', 'heinrich cornelius agrippa']]
question
by what century was gunpowder used by the russians ?

distractors
[['visual', 'spanish', 'their']]
question

distractors
[['NA']]
question

distractors
[['NA']]
topic: Astronomer
number of paragraphs: 9
question



distractors
[['NA'], ['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[[

[['twins', 'trouble', 'characters']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
who dedicated apollo -lrb- ?

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
who trained the most of apollo ?

distractors
[['vaccine hypotheses', 'autism spectrum disorder', 'corner']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
who descended to apollo from the beginning to apollo ?

distractors
[['founder', 'reign', 'play']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
who was the god of music with apollo ?

distractors
[['hand', 'years', 'bandwidth']]
question
who advocated orestes rules ?

distractors
[['orwell', 'apollo', 'hittite']]
question

distractors
[['NA']]
q

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
where was the olympic first held ?


where is the rugby team based ?

distractors
[['NA'], ['copa constitució', 'tribunal de corts', 'henry iv']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question
what is the population of galicia ?


what is the parliament of suriname known as ?

distractors
[['american football league', 'france', 'national football league'], ['france', 'firearm', 'south']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA']]
question

distractors
[['NA'

### Old Function

In [134]:
for i in range(len(sample100)):
    try:
        print('Sample: ' + str(i))
        print('Raw Answer: ' + str(sample100.loc[i]['answers']))
        answer_type = check_answer_type(sample100.loc[i]['answers'])
        print('Answer type: ' + str(answer_type))
        topic, paragraph, question, correct_answer, answer_type, wrong_answers = generate_distractor(sample100.loc[i]['topics'], sample100.loc[i]['paragraphs'], \
                                  sample100.loc[i]['questions'], sample100.loc[i]['answers'], answer_type)
        print('---------------------------------------------------------')
        print('Question: ' + str(question))
        print('Wrong answers: ' + str(wrong_answers))
        print('\n')
    except:
        print("error occurred for sample#: " + str(i))

Sample: 0
Raw Answer: North Carolina and New Mexico
Answer type: ['PROPN']
---------------------------------------------------------
Question: In what states are pharmacist clinicians given prescriptive and diagnostic authority?
Wrong answers: ['indian health service', 'north carolina', 'new mexico']


Sample: 1
Raw Answer: non-specific
Answer type: ['ADJ']
---------------------------------------------------------
Question: The innate immune system responds in a generic way, meaning it is what?
Wrong answers: ['its great', 'innate immune', 'long-lasting']


Sample: 2
Raw Answer: charter
Answer type: ['NOUN']
---------------------------------------------------------
Question: Granting what status would allow private non-religious schools in the US to receive public funds?
Wrong answers: ['list', 'constitution', 'economy']


Sample: 3
Raw Answer: pigment-filled plastids responsible for the bright colors seen in flowers and ripe fruit
Answer type: ['ADJ']
---------------------------------

Answer type: ['ADJ']
---------------------------------------------------------
Question: What health problem did Tesla have in 1879?
Wrong answers: ['subsequent claims', 'same time', 'his family']


Sample: 36
Raw Answer: four
Answer type: ['NUM']
---------------------------------------------------------
Question: How many auricles do most species have?
Wrong answers: ['2', '8', 'two']


Sample: 37
Raw Answer: Supreme Court of the United States
Answer type: ['PROPN']
---------------------------------------------------------
Question: A decision made by what entity restored Tesla's patents?
Wrong answers: ['united states', 'oliver lodge', 'the united states']


Sample: 38
Raw Answer: Charles F. Peck
Answer type: ['PROPN']
---------------------------------------------------------
Question: Who else did Tesla make the acquaintance of in 1886?
Wrong answers: ['hotel new yorker', 'tesla electric company', 'petroleum exporting countries']


Sample: 39
Raw Answer: second most commonly
Answer 

error occurred for sample#: 66
Sample: 67
Raw Answer: Kenya's various ethnic groups typically speak their mother tongues within their own communities
Answer type: ['PROPN']
---------------------------------------------------------
Question: What language is spoken in Kenya?
Wrong answers: ['kenya', 'their mother tongues', 'armed forces']


Sample: 68
Raw Answer: flattened circular
Answer type: ['VP']
---------------------------------------------------------
Question: What shape are granal thylakoids?
Wrong answers: ['are continuous', 'retained fundamental', 'was due']


Sample: 69
Raw Answer: artifact
Answer type: ['ADJ']
---------------------------------------------------------
Question: What is the force called rgarding a potential field between two locations?
Wrong answers: ['british', 'earliest', 'civil']


Sample: 70
Raw Answer: force
Answer type: ['NOUN']
---------------------------------------------------------
Question: What notion keeps it's meaning through both Netonian and S

---------------------------------------------------------
Question: Los Angeles is in the lower part of what?
Wrong answers: ['serbian cyrillic', 'southern california', 'никола']


Sample: 99
Raw Answer: Veni redemptor gentium
Answer type: ['NOUN']
---------------------------------------------------------
Question: What was the hymn based on?
Wrong answers: ['oil shock', 'term effects', 'redemptor gentium']




In [183]:
# For Debugging
topic, paragraph, question, correct_answer, answer_type, wrong_answers = generate_distractor(sample100.loc[71]['topics'], sample100.loc[71]['paragraphs'], \
                                  sample100.loc[71]['questions'], sample100.loc[71]['answers'], check_answer_type(sample100.loc[71]['answers']))
print(topic, paragraph, question, correct_answer, answer_type, wrong_answers)

In [70]:
def long_generate_distractor(topic, paragraph, question, answer, answer_type):
    wrong_answers=[]
    #preprocess answer
    correct_answer = nlp(preprocess(answer))
    # get answer pos
    ans_length = len(correct_answer)
    ans_tag = [token.tag_ for token in correct_answer]
    ans_pos = [token.pos_ for token in correct_answer]
    # tokenize paragraph
    article = nlp(unidecode.unidecode(paragraph))
    doc = textacy.Doc(paragraph, lang='en_core_web_sm')
        
    # Preprocessing for same sentence distractor generation
    # get all sentences in paragraph
    sent_detector = nltk.data.load('tokenizers/punkt/english.pickle')
    sent_list = sent_detector.tokenize(paragraph.strip())

    # find sentence that has answer
    for s in sent_list:
        if answer in s:
            ans_sent = s
    # tokenize sentence
    sentence = nlp(ans_sent)
    sent = textacy.Doc(sentence, lang='en_core_web_sm')
        
    # Preprocessing for same topic distractor generation
    # choose random paragraph from same topic
    topic_index = next((index for (index, d) in enumerate(df['data']) if d["title"] == topic), None)
    index_list = list(range(len(df['data'][topic_index]['paragraphs'])))
    # make sure isn't the same paragraph as current paragraph
    p_index = next((index for (index, d) in enumerate(df['data'][topic_index]['paragraphs'])\
                    if d["context"] == paragraph), None)
    index_list.remove(p_index)
    alt_p_index = random.choice(index_list)
    alt_p = df['data'][topic_index]['paragraphs'][alt_p_index]['context']
    alt_article = nlp(alt_p)
    alt_paragraph = textacy.Doc(alt_p, lang='en_core_web_sm')
        
    # Preprocessing for different topic distractor generation
    # choose random topic
    topic_list = list(range(len(df['data'])))
    # make sure isn't the same topic as current topic
    topic_list.remove(topic_index)
    alt_topic_index = random.choice(topic_list)
    alt_topic = df['data'][alt_topic_index]['title']
    alt_topic_paragraph = df['data'][alt_topic_index]['paragraphs'][0]['context']
    # select first paragraph from topic - option to randomize selection as well
    alt_topic_article = nlp(alt_topic_paragraph)
    alt_topic = textacy.Doc(alt_topic_paragraph, lang='en_core_web_sm')
    
    if 'VP' in answer_type:
        vb_pattern = r'<VERB>+<ADJ>*<ADV>*<PART>*<DET>*<NOUN>+<ADP>*<DET>*<NOUN>*'
        vb_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(doc, vb_pattern)]
#         print('vb_list: ' + str(vb_list))
        sent_vb_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(sent, vb_pattern)]
#         print('sent_vb_list: ' + str(sent_vb_list))
        alt_paragraph_vb_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_paragraph, vb_pattern)]
#         print('alt_paragraph_vb_list: ' + str(alt_paragraph_vb_list))
        alt_topic_vb_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_topic, vb_pattern)]  
#         print('alt_topic_vb_list: ' + str(alt_topic_vb_list))
        merged = set(vb_list+sent_vb_list+alt_paragraph_vb_list+alt_topic_vb_list)
        for i in merged:
#             print('i: ' + str(i))
            item = nlp(i)
#             print('item[0].tag_ '+str(item[0].tag_))
#             print('ans_tag[0] '+str(ans_tag[0]))
#             print('item[-1].pos_ '+str(item[-1].pos_))
#             print('ans_pos[-1] '+str(ans_pos[-1]))
            if (item[0].tag_ == ans_tag[0]) and (item[-1].pos_ == ans_pos[-1]):
                wrong_answers.append(item)
        if len(wrong_answers) < 3:
                wrong_answers = merged
    elif 'VERB' in answer_type:
        verbs = [token.orth_.lower() for token in article if (token.pos_=='VERB')]
        sent_verbs = verbs = [token.orth_.lower() for token in sentence if (token.pos_=='VERB')]
        alt_paragraph_verbs = [token.orth_.lower() for token in alt_article if (token.pos_=='VERB')]
        alt_topic_verbs = [token.orth_.lower() for token in alt_topic_article if (token.pos_=='VERB')]
        merged = set(verbs+sent_verbs+alt_paragraph_verbs+alt_topic_verbs)
        for i in merged:
            item = nlp(i)
            if (item[0].tag_ == ans_pos_tag[0]):
                wrong_answers.append(item)
    elif 'ADJ' in answer_type:
        if ans_length==1:
            single_adj = [token.orth_.lower() for token in article if token.pos_=='ADJ']
            single_adj_sent = [token.orth_.lower() for token in sentence if token.pos_=='ADJ']
            alt_paragraph_single_adj = [token.orth_.lower() for token in alt_article if token.pos_=='ADJ']
            alt_topic_single_adj = [token.orth_.lower() for token in alt_topic_article if token.pos_=='ADJ']
            wrong_answers = set(single_adj+single_adj_sent+alt_paragraph_single_adj+alt_topic_single_adj)
        else:
            ent_list = [str(i) for i in list(article.ents)]
#             print(ent_list)
            if answer in ent_list:
                ent_labels = [x.label_ for x in article.ents]
                # get all named entities in sentence
                sent_ent_list = [str(i) for i in list(sentence.ents)]
                sent_labels = [x.label_ for x in sentence.ents]
                alt_topic_ent_list = [str(i) for i in list(alt_topic_article.ents)]
                alt_topic_labels = [x.label_ for x in alt_topic_article.ents]
                alt_article_ent_list = [str(i) for i in list(alt_article.ents)]
                alt_article_labels = [x.label_ for x in alt_article.ents]

                # create table of named entities
                max_length = max(len(ent_list), len(sent_ent_list), len(alt_article_ent_list),len(alt_topic_ent_list))
        #         print(max_length)
                ne_pd = pd.DataFrame()
                ne_pd['entity'] = ent_list + (['NA'] * (max_length - len(ent_list)))
                ne_pd['label'] = list(ent_labels) + (['NA'] * (max_length - len(ent_list)))
                ne_pd['sent_entity'] = sent_ent_list + (['NA'] * (max_length - len(sent_ent_list)))
                ne_pd['sent_label'] = list(sent_labels) + (['NA'] * (max_length - len(sent_ent_list)))
                ne_pd['altp_entity'] = alt_article_ent_list + (['NA'] * (max_length - len(alt_article_ent_list)))
                ne_pd['altp_label'] = list(alt_article_labels) + (['NA'] * (max_length - len(alt_article_ent_list)))
                ne_pd['altt_entity'] = alt_topic_ent_list + (['NA'] * (max_length - len(alt_topic_ent_list)))
                ne_pd['altt_label'] = list(alt_topic_labels) + (['NA'] * (max_length - len(alt_topic_ent_list)))

                ans_label = [ne_pd[ne_pd['entity']==e]['label'].values[0] for e in ne_pd['entity'] if e in answer]
        #         print(ans_label)
                alt_ans_list = list(ne_pd[(ne_pd['label'].isin(ans_label))]['entity'])
                alt_ans_list.extend(list(ne_pd[(ne_pd['sent_label'].isin(ans_label))]['sent_entity']))
                alt_ans_list.extend(list(ne_pd[(ne_pd['altp_label'].isin(ans_label))]['altp_entity']))
                alt_ans_list.extend(list(ne_pd[(ne_pd['altt_label'].isin(ans_label))]['altt_entity']))
                wrong_answers = set(alt_ans_list)
            else:
            #adj_pattern = r'<ADV>*<ADJ>+<PART>*<DET>*<NOUN>*'
                correct_ans_pos = str(['r'+str(token.pos_)+'l' for token in \
               nlp(answer)])[1:-1].replace("'r","<").replace("l'",">").replace(',','+',1).replace(',','*').replace(' ','')
                adj_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(doc, correct_ans_pos)]
                sent_adj_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(sent, correct_ans_pos)]
                alt_paragraph_adj_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_paragraph, correct_ans_pos)]
                alt_topic_adj_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_topic, correct_ans_pos)]

                hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),paragraph)
                hyphen_sep_words_sent = re.findall(r'\w+(?:-\w+)+'.lower(),ans_sent)
                alt_p_hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),alt_p)
                alt_topic_hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),alt_topic_paragraph)

                adj_list.extend(hyphen_sep_words)
                sent_adj_list.extend(hyphen_sep_words_sent)
                alt_paragraph_adj_list.extend(alt_p_hyphen_sep_words)
                alt_topic_adj_list.extend(alt_topic_hyphen_sep_words)
                wrong_answers = set(adj_list+sent_adj_list+alt_paragraph_adj_list+alt_topic_adj_list)
    elif 'ADV' in answer_type:
        if ans_length==1:
            single_adj = [token.orth_.lower() for token in article if token.pos_=='ADV']
            single_adj_sent = [token.orth_.lower() for token in sentence if token.pos_=='ADV']
            alt_paragraph_single_adj = [token.orth_.lower() for token in alt_article if token.pos_=='ADV']
            alt_topic_single_adj = [token.orth_.lower() for token in alt_topic_article if token.pos_=='ADV']
            wrong_answers = set(single_adj+single_adj_sent+alt_paragraph_single_adj+alt_topic_single_adj)
        else:
            ent_list = [str(i) for i in list(article.ents)]
#             print(ent_list)
            if answer in ent_list:
                ent_labels = [x.label_ for x in article.ents]
                # get all named entities in sentence
                sent_ent_list = [str(i) for i in list(sentence.ents)]
                sent_labels = [x.label_ for x in sentence.ents]
                alt_topic_ent_list = [str(i) for i in list(alt_topic_article.ents)]
                alt_topic_labels = [x.label_ for x in alt_topic_article.ents]
                alt_article_ent_list = [str(i) for i in list(alt_article.ents)]
                alt_article_labels = [x.label_ for x in alt_article.ents]

                # create table of named entities
                max_length = max(len(ent_list), len(sent_ent_list), len(alt_article_ent_list),len(alt_topic_ent_list))
        #         print(max_length)
                ne_pd = pd.DataFrame()
                ne_pd['entity'] = ent_list + (['NA'] * (max_length - len(ent_list)))
                ne_pd['label'] = list(ent_labels) + (['NA'] * (max_length - len(ent_list)))
                ne_pd['sent_entity'] = sent_ent_list + (['NA'] * (max_length - len(sent_ent_list)))
                ne_pd['sent_label'] = list(sent_labels) + (['NA'] * (max_length - len(sent_ent_list)))
                ne_pd['altp_entity'] = alt_article_ent_list + (['NA'] * (max_length - len(alt_article_ent_list)))
                ne_pd['altp_label'] = list(alt_article_labels) + (['NA'] * (max_length - len(alt_article_ent_list)))
                ne_pd['altt_entity'] = alt_topic_ent_list + (['NA'] * (max_length - len(alt_topic_ent_list)))
                ne_pd['altt_label'] = list(alt_topic_labels) + (['NA'] * (max_length - len(alt_topic_ent_list)))

                ans_label = [ne_pd[ne_pd['entity']==e]['label'].values[0] for e in ne_pd['entity'] if e in answer]
        #         print(ans_label)
                alt_ans_list = list(ne_pd[(ne_pd['label'].isin(ans_label))]['entity'])
                alt_ans_list.extend(list(ne_pd[(ne_pd['sent_label'].isin(ans_label))]['sent_entity']))
                alt_ans_list.extend(list(ne_pd[(ne_pd['altp_label'].isin(ans_label))]['altp_entity']))
                alt_ans_list.extend(list(ne_pd[(ne_pd['altt_label'].isin(ans_label))]['altt_entity']))
                wrong_answers = set(alt_ans_list)
            else:
            #adj_pattern = r'<ADV>*<ADJ>+<PART>*<DET>*<NOUN>*'
                correct_ans_pos = str(['r'+str(token.pos_)+'l' for token in \
               nlp(answer)])[1:-1].replace("'r","<").replace("l'",">").replace(',','+',1).replace(',','*').replace(' ','')
                adj_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(doc, correct_ans_pos)]
                sent_adj_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(sent, correct_ans_pos)]
                alt_paragraph_adj_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_paragraph, correct_ans_pos)]
                alt_topic_adj_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_topic, correct_ans_pos)]

                hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),paragraph)
                hyphen_sep_words_sent = re.findall(r'\w+(?:-\w+)+'.lower(),ans_sent)
                alt_p_hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),alt_p)
                alt_topic_hyphen_sep_words = re.findall(r'\w+(?:-\w+)+'.lower(),alt_topic_paragraph)

                adj_list.extend(hyphen_sep_words)
                sent_adj_list.extend(hyphen_sep_words_sent)
                alt_paragraph_adj_list.extend(alt_p_hyphen_sep_words)
                alt_topic_adj_list.extend(alt_topic_hyphen_sep_words)
                wrong_answers = set(adj_list+sent_adj_list+alt_paragraph_adj_list+alt_topic_adj_list)
    elif 'NOUN' in answer_type:
        if ans_length==1:
            single_noun = [token.orth_.lower() for token in article if (token.pos_=='NOUN')]
            single_noun_sent = [token.orth_.lower() for token in sentence if (token.pos_=='NOUN')]
            alt_single_noun = [token.orth_.lower() for token in alt_article if (token.pos_=='NOUN')]
            alt_topic_single_noun = [token.orth_.lower() for token in alt_topic_article if (token.pos_=='NOUN')]
            wrong_answers = set(single_noun+single_noun_sent+alt_single_noun+alt_topic_single_noun)
        else:
            ent_list = [str(i) for i in list(article.ents)]
#             print(ent_list)
            if answer in ent_list:
                ent_labels = [x.label_ for x in article.ents]
                # get all named entities in sentence
                sent_ent_list = [str(i) for i in list(sentence.ents)]
                sent_labels = [x.label_ for x in sentence.ents]
                alt_topic_ent_list = [str(i) for i in list(alt_topic_article.ents)]
                alt_topic_labels = [x.label_ for x in alt_topic_article.ents]
                alt_article_ent_list = [str(i) for i in list(alt_article.ents)]
                alt_article_labels = [x.label_ for x in alt_article.ents]

                # create table of named entities
                max_length = max(len(ent_list), len(sent_ent_list), len(alt_article_ent_list),len(alt_topic_ent_list))
        #         print(max_length)
                ne_pd = pd.DataFrame()
                ne_pd['entity'] = ent_list + (['NA'] * (max_length - len(ent_list)))
                ne_pd['label'] = list(ent_labels) + (['NA'] * (max_length - len(ent_list)))
                ne_pd['sent_entity'] = sent_ent_list + (['NA'] * (max_length - len(sent_ent_list)))
                ne_pd['sent_label'] = list(sent_labels) + (['NA'] * (max_length - len(sent_ent_list)))
                ne_pd['altp_entity'] = alt_article_ent_list + (['NA'] * (max_length - len(alt_article_ent_list)))
                ne_pd['altp_label'] = list(alt_article_labels) + (['NA'] * (max_length - len(alt_article_ent_list)))
                ne_pd['altt_entity'] = alt_topic_ent_list + (['NA'] * (max_length - len(alt_topic_ent_list)))
                ne_pd['altt_label'] = list(alt_topic_labels) + (['NA'] * (max_length - len(alt_topic_ent_list)))

                ans_label = [ne_pd[ne_pd['entity']==e]['label'].values[0] for e in ne_pd['entity'] if e in answer]
        #         print(ans_label)
                alt_ans_list = list(ne_pd[(ne_pd['label'].isin(ans_label))]['entity'])
                alt_ans_list.extend(list(ne_pd[(ne_pd['sent_label'].isin(ans_label))]['sent_entity']))
                alt_ans_list.extend(list(ne_pd[(ne_pd['altp_label'].isin(ans_label))]['altp_entity']))
                alt_ans_list.extend(list(ne_pd[(ne_pd['altt_label'].isin(ans_label))]['altt_entity']))
                wrong_answers = set(alt_ans_list)
            else:
                correct_ans_pos = str(['r'+str(token.pos_)+'l' for token in \
               nlp(answer)])[1:-1].replace("'r","<").replace("l'",">").replace(',','+',1).replace(',','*').replace(' ','')
                np_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(doc, correct_ans_pos)]
                sent_np_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(sent, correct_ans_pos)]
                alt_paragraph_np_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_paragraph, correct_ans_pos)]
                alt_topic_np_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_topic, correct_ans_pos)]
                wrong_answers = set(np_list+sent_np_list+alt_paragraph_np_list+alt_topic_np_list)
    elif 'YEAR' in answer_type:
        number_list = [token.orth_ for token in article if token.pos_=='NUM']
        number_list_sent = [token.orth_ for token in sentence if token.pos_=='NUM']
        alt_paragraph_number_list = [token.orth_ for token in alt_article if token.pos_=='NUM']
        alt_topic_number_list = [token.orth_ for token in alt_topic_article if token.pos_=='NUM']
        merged = set(number_list+number_list_sent+alt_paragraph_number_list+alt_topic_number_list)
        for i in merged:
            if len(i)==4:
                wrong_answers.append(i)
    elif 'NUM' in answer_type:
        number_list = [token.orth_ for token in article if token.pos_=='NUM']
        number_list_sent = [token.orth_ for token in sentence if token.pos_=='NUM']
        alt_paragraph_number_list = [token.orth_ for token in alt_article if token.pos_=='NUM']
        alt_topic_number_list = [token.orth_ for token in alt_topic_article if token.pos_=='NUM']
        merged = set(number_list+number_list_sent+alt_paragraph_number_list+alt_topic_number_list)
        wrong_answers = [t2d.convert(str(i)).lstrip() for i in merged]
    elif 'TIME' in answer_type:
        time_list = [token.orth_ for token in article if token.pos_=='NUM']
        time_list_sent = [token.orth_ for token in sentence if token.pos_=='NUM']
        alt_paragraph_time_list = [token.orth_ for token in alt_article if token.pos_=='NUM']
        alt_topic_time_list = [token.orth_ for token in alt_topic_article if token.pos_=='NUM']
        merged = set(time_list+time_list_sent+alt_paragraph_time_list+alt_topic_time_list)
        print('merged: ' + str(merged))
        wrong_answers = [t2d.convert(str(i)).lstrip()+' ' + str(correct_answer[-1]) for i in merged]
    elif 'MONEY' in answer_type:
        currency = answer[0]
        number_list = [str(currency) + token.orth_ for token in article if token.pos_=='NUM']
        number_list_sent = [str(currency) + token.orth_ for token in sentence if token.pos_=='NUM']
        alt_paragraph_number_list = [str(currency) + token.orth_ for token in alt_article if token.pos_=='NUM']
        alt_topic_number_list = [str(currency) + token.orth_ for token in alt_topic_article if token.pos_=='NUM']
        merged = set(number_list+number_list_sent+alt_paragraph_number_list+alt_topic_number_list)
        wrong_answers = [t2d.convert(str(i)).replace(' ','') for i in merged]
    elif 'DATE' in answer_type:
        if ans_length==1:
            month_list=random.sample(['January', 'Jan', 'February', 'March', 'April', \
                                   'May','June', 'July', 'August', 'September', \
                                   'October','November', \
                                      'December'].remove(answer.capitalize()),3)
        else:
            date_regex = r'<NUM>*<PROPN>+<NUM>*<PUNCT>*<NUM>+'
            date_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(doc, date_regex)]
            sent_date_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(sent, date_regex)]
            alt_paragraph_date_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_paragraph, date_regex)]
            alt_topic_date_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_topic, date_regex)]
            wrong_answers = set(date_list+sent_date_list+alt_paragraph_date_list+alt_topic_date_list)
    elif 'PROPN' in answer_type:
        ent_list = [str(i) for i in list(article.ents)]
#             print(ent_list)
        if answer in ent_list:
            ent_labels = [x.label_ for x in article.ents]
            # get all named entities in sentence
            sent_ent_list = [str(i) for i in list(sentence.ents)]
            sent_labels = [x.label_ for x in sentence.ents]
            alt_topic_ent_list = [str(i) for i in list(alt_topic_article.ents)]
            alt_topic_labels = [x.label_ for x in alt_topic_article.ents]
            alt_article_ent_list = [str(i) for i in list(alt_article.ents)]
            alt_article_labels = [x.label_ for x in alt_article.ents]

            # create table of named entities
            max_length = max(len(ent_list), len(sent_ent_list), len(alt_article_ent_list),len(alt_topic_ent_list))
    #         print(max_length)
            ne_pd = pd.DataFrame()
            ne_pd['entity'] = ent_list + (['NA'] * (max_length - len(ent_list)))
            ne_pd['label'] = list(ent_labels) + (['NA'] * (max_length - len(ent_list)))
            ne_pd['sent_entity'] = sent_ent_list + (['NA'] * (max_length - len(sent_ent_list)))
            ne_pd['sent_label'] = list(sent_labels) + (['NA'] * (max_length - len(sent_ent_list)))
            ne_pd['altp_entity'] = alt_article_ent_list + (['NA'] * (max_length - len(alt_article_ent_list)))
            ne_pd['altp_label'] = list(alt_article_labels) + (['NA'] * (max_length - len(alt_article_ent_list)))
            ne_pd['altt_entity'] = alt_topic_ent_list + (['NA'] * (max_length - len(alt_topic_ent_list)))
            ne_pd['altt_label'] = list(alt_topic_labels) + (['NA'] * (max_length - len(alt_topic_ent_list)))

            ans_label = [ne_pd[ne_pd['entity']==e]['label'].values[0] for e in ne_pd['entity'] if e in answer]
    #         print(ans_label)
            alt_ans_list = list(ne_pd[(ne_pd['label'].isin(ans_label))]['entity'])
            alt_ans_list.extend(list(ne_pd[(ne_pd['sent_label'].isin(ans_label))]['sent_entity']))
            alt_ans_list.extend(list(ne_pd[(ne_pd['altp_label'].isin(ans_label))]['altp_entity']))
            alt_ans_list.extend(list(ne_pd[(ne_pd['altt_label'].isin(ans_label))]['altt_entity']))
            wrong_answers = set(alt_ans_list) - set([str(correct_answer)])
        else:
            correct_ans_pos = str(['r'+str(token.pos_)+'l' for token in \
           nlp(answer)])[1:-1].replace("'r","<").replace("l'",">").replace(',','+',1).replace(',','*').replace(' ','')
            np_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(doc, correct_ans_pos)]
            sent_np_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(sent, correct_ans_pos)]
            alt_paragraph_np_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_paragraph, correct_ans_pos)]
            alt_topic_np_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_topic, correct_ans_pos)]
            wrong_answers = set(np_list+sent_np_list+alt_paragraph_np_list+alt_topic_np_list)
    else:
        correct_ans_pos = str(['r'+str(token.pos_)+'l' for token in \
           nlp(answer)])[1:-1].replace("'r","<").replace("l'",">").replace(',','+',1).replace(',','*').replace(' ','')
        propn_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(doc, correct_ans_pos)]
        sent_propn_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(sent, correct_ans_pos)]
        alt_paragraph_propn_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_paragraph, correct_ans_pos)]
        alt_topic_propn_list = [l.text.lower() for l in textacy.extract.pos_regex_matches(alt_topic, correct_ans_pos)]
        wrong_answers = set(propn_list+sent_propn_list+alt_paragraph_propn_list+alt_topic_propn_list)
        if 'ADP' in answer_type:
            wrong_answers = [i for i in wrong_answers if i[0]==correct_answer[0].orth_]
    wrong_answers = [i for i in wrong_answers for word in range(len(correct_answer)) if correct_answer[word].orth_ not in i]
                
    return (topic, paragraph, question, correct_answer, answer_type, set(wrong_answers))
        